# Final ROSCOE scoring of CausalBench

## Merging datasets 

### no gt

In [126]:
# common imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import re
from typing import Union, List, Tuple, Dict, Any, Optional

In [127]:
# read two cvs files
overall_result = pd.read_csv("raw_datasets/bern_cna_35_gpt4cot.csv")
overall_result.head()

,ID,descriptive_id,sensical,query_type,rung,phenomenon,simpson,truth,prompt,truth_norm,pred
0,4669,confounding_a_ett_gender_pay,-1,ett,3,confounding,False,yes,"Imagine a self-contained, hypothetical world w...",1,No
1,4046,collision_n_nie_nonsense0,0,nie,3,collision,False,no,"Imagine a self-contained, hypothetical world w...",0,No
2,1597,arrowhead_c_ett_floor_wet,1,ett,3,arrowhead,False,yes,"Imagine a self-contained, hypothetical world w...",1,Yes
3,2995,chain_a_nie_smoking_gene_cancer,-1,nie,3,chain,False,yes,"Imagine a self-contained, hypothetical world w...",1,No
4,9475,nondet-diamondcut_c_nde_smoking_frontdoor,1,nde,3,nondet-diamondcut,False,yes,"Imagine a self-contained, hypothetical world w...",1,Yes


In [128]:
# keep just ID and prompt columns
overall_result = overall_result[["ID", "prompt"]]
overall_result.head()

,ID,prompt
0,4669,"Imagine a self-contained, hypothetical world w..."
1,4046,"Imagine a self-contained, hypothetical world w..."
2,1597,"Imagine a self-contained, hypothetical world w..."
3,2995,"Imagine a self-contained, hypothetical world w..."
4,9475,"Imagine a self-contained, hypothetical world w..."


In [129]:
prompt2gpt_response_lookup = pd.read_csv("raw_datasets/cache_gpt4_responses.csv")
prompt2gpt_response_lookup.head()

,pred,query
0,"No, the prisoner being alive cannot be guarant...",We know that captain commands causes rifleman1...
1,pred,query
2,"No, we cannot definitively conclude that the s...",principal's direction makes teacher1 and teach...
3,"No, based on the information provided, we cann...",principal's anger makes teacher1 and teacher2 ...
4,"No, the student might not necessarily be alive...",We know that captain commands causes professor...


In [130]:
# print full contennt of thrid row
print(prompt2gpt_response_lookup.iloc[3, 1])

principal's anger makes teacher1 and teacher2 to blame the student. If either teacher1 of teacher2 blame the student, the student become emotionally damaged. We observed the principal wasn't angry. Would the student be emotionally damaged if teacher2 blame the student?  
Start your answer with "Yes", or "No", followed by step-by-step reasoning to support your explanation.


In [131]:
# Using the "prompt" from overall_result df, look up the GPT4’s reasoning chain in the .cache_gpt4_responses.csv file (use "query" to look up "pred").
# If the "pred" is not empty, then add the "pred" to the overall_result df.

overall_result["pred"] = overall_result["prompt"].apply(
    lambda x: prompt2gpt_response_lookup[prompt2gpt_response_lookup["query"] == x][
        "pred"
    ].values[0]
    if len(
        prompt2gpt_response_lookup[prompt2gpt_response_lookup["query"] == x][
            "pred"
        ].values
    )
    > 0
    else np.nan
)
overall_result.head()

,ID,prompt,pred
0,4669,"Imagine a self-contained, hypothetical world w...",Step 1) Extract the causal graph: The causal g...
1,4046,"Imagine a self-contained, hypothetical world w...",Step 1) Extract the causal graph: The causal g...
2,1597,"Imagine a self-contained, hypothetical world w...",Step 1) Extract the causal graph: The causal g...
3,2995,"Imagine a self-contained, hypothetical world w...",Step 1) Extract the causal graph: The causal g...
4,9475,"Imagine a self-contained, hypothetical world w...",Step 1) Extract the causal graph: The causal g...


In [132]:
# print full contennt of first row
print(overall_result.iloc[45, 1])
print("#" * 100)
print(overall_result.iloc[45, 2])

Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Male gender has a direct effect on non-competitive department application and in-state residency. Non-competitive department application has a direct effect on admission. In-state residency has a direct effect on admission. For individuals who are not male, the probability of admission is 64.83%. For individuals who are male, the probability of admission is 43.35%. For individuals who are male, would it be more likely to see admission if the individual had not been male?

Hint: You can answer the question by following the subquestions below:

Step 1) Extract the causal graph: What is the causal graph expressed in the context? Use "X" to denote "gender". Use "V2" to denote "department applying to". Use "V3" to denote "residency status". Use "Y" to denote "admission status". Answer nothing else but each edge one by one, in the format of "var1 -> va

In [133]:
# print len of overall_result
print(len(overall_result))
# new df with no null values
overall_result_no_null = overall_result.dropna()
# print len of new df
print(len(overall_result_no_null))

612
612


In [134]:
# Check if there are two identical entries in pred column
overall_result_no_null["pred"].value_counts()

Step 1) Extract the causal graph: The causal graph expressed in the context is "X -> V2, V2 -> Y".\n\nStep 2) Identify the query type: The query type of the above question is "natural direct effect".\n\nStep 3) Translate the query to an estimand: Based on the type of the causal query, the question can be translated to a formal estimand as E(Y_x | do(V2)).\n\nStep 4) Collect all the available data: We do not have any specific data provided in the question. However, we know the causal relationships: P(X -> V2) and P(V2 -> Y).\n\nStep 5) Solve for the estimand: Given all the information above, we can solve for the estimand E(Y_x | do(V2)) using causal inference skills. Since we want to understand how the gene affects lung cancer, we need to consider the effect of the gene on smoking status and the effect of smoking status on lung cancer. Therefore, we should look at the correlation case by case according to gene, smoking status, and lung cancer. So, the answer is "No".                    

In [135]:
# Check if there are two identical entries in prompt column
overall_result_no_null["prompt"].value_counts()

Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Rujx has a direct effect on snov. Nyrp has a direct effect on snov.  If we disregard the mediation effect through snov, would rujx still positively affect nyrp?\n\nHint: You can answer the question by following the subquestions below:\n\nStep 1) Extract the causal graph: What is the causal graph expressed in the context? Use "X" to denote "rujx". Use "Y" to denote "nyrp". Use "V3" to denote "snov". Answer nothing else but each edge one by one, in the format of "var1 -> var2", and use "," to separate the edges.\n\nStep 2) Identify the query type: What is the query type of the above question? Choose one from "marginal probability", "conditional probability", "explaining away effect", "backdoor adjustment set", "average treatment effect", "collider bias", "normal counterfactual question", "average treatment effect on treated", "natural direct effect

### With GT

In [56]:
# Helper function to search a nested dictionary
def search_nested_dict(nested_dict, target_key):
    queue = [(nested_dict, key) for key in nested_dict]

    while queue:
        current_dict, key = queue.pop(0)

        if key == target_key:
            return current_dict[key]

        if isinstance(current_dict[key], type({})):
            for sub_key in current_dict[key]:
                queue.append((current_dict[key], sub_key))
    return None

In [115]:
# Read txt file
with open("raw_datasets/ground_truth_reasoning_template.txt", "r", encoding='utf-8') as file:
    reasoning_template = file.read()
print(reasoning_template)

Step 1) Extract the causal graph: The causal graph expressed in the context is: "{graph}".

Step 2) Identify the query type: The query type of the above question is "{query_type}".

Step 3) Translate the query to an estimand: Based on the type of the causal query, the question can be translated to a formal estimand as: "{step1}".

Step 4) Collect all the available data: The available data are: "{given_info_formal}".

Step 5) Solve for the estimand: To solve for the estimand, we need to compute {step1_format}.

Using the causal graph, we can simplify the estimand to
{step1}
= {step2}

Using the available data, we can calculate:
{step2}
= {step3}
≈ {step4}

Since the estimate for the estimand is {step4}, the overall answer to the question is "{answer}".





In [97]:
# Get formatting keys of reasoning template
reasoning_template_keys = re.findall(r"\{(.+?)\}", reasoning_template)
reasoning_template_keys

['graph',
 'query_type',
 'step1',
 "'; '.join(given_info_formal)",
 "step1.replace(' - ', ' and ')",
 'step1',
 'step2',
 'step2',
 'step3',
 'step4',
 'step4',
 'answer']

In [65]:
# Load the json file as a list of nested dictionary
import json
with open("raw_datasets/bern_cna_35.json") as f:
    nested_dicts = json.load(f)

<class 'list'>
<class 'dict'>
dict_keys(['ID', 'descriptive_id', 'sensical', 'story_id', 'background', 'given_info', 'question', 'variable_mapping', 'answer', 'meta', 'reasoning', 'given_info_formal', 'given_info_para'])


In [85]:
# Create df with ID comlumn of same lenght as nested_dict
df = pd.DataFrame({"ID": [i for i in range(len(nested_dict))]})
# Extract query_type, graph, step0, step1, step2, step3, step4, answer, given_info_formal, from nested_dict and add them to df
df["query_type"] = df["ID"].apply(lambda x: search_nested_dict(nested_dict[x], "query_type"))
df["graph"] = df["ID"].apply(lambda x: search_nested_dict(nested_dict[x], "graph"))
df["step0"] = df["ID"].apply(lambda x: search_nested_dict(nested_dict[x], "step0")) 
df["step1"] = df["ID"].apply(lambda x: search_nested_dict(nested_dict[x], "step1"))
df["step2"] = df["ID"].apply(lambda x: search_nested_dict(nested_dict[x], "step2"))
df["step3"] = df["ID"].apply(lambda x: search_nested_dict(nested_dict[x], "step3"))
df["step4"] = df["ID"].apply(lambda x: search_nested_dict(nested_dict[x], "step4"))
df["answer"] = df["ID"].apply(lambda x: search_nested_dict(nested_dict[x], "answer"))
df["given_info_formal"] = df["ID"].apply(lambda x: search_nested_dict(nested_dict[x], "given_info_formal"))
df.head()

,ID,query_type,graph,step0,step1,step2,step3,step4,answer,given_info_formal
0,0,ate,"V1->X,V1->Y,V2->X,X->Y",E[Y|do(X = 1)] − E[Y|do(X = 0)],P(Y=1|do(X=1)) - P(Y=1|do(X=1)),[P(Y=1|V2=1)-P(Y=1|V2=0)]/[P(X=1|V2=1)-P(X=1|V...,[0.5671622485-0.6026637459]/[0.2-0.14],-0.5917,no,"[P(Y=1|V2=0)=0.6026637459, P(Y=1|V2=1)=0.56716..."
1,1,ate,"V1->X,V1->Y,V2->X,X->Y",E[Y|do(X = 1)] − E[Y|do(X = 0)],P(Y=1|do(X=1)) - P(Y=1|do(X=1)),[P(Y=1|V2=1)-P(Y=1|V2=0)]/[P(X=1|V2=1)-P(X=1|V...,[0.39733441500000005-0.44394854770000003]/[0.9...,-0.1227,no,"[P(Y=1|V2=0)=0.44394854770000003, P(Y=1|V2=1)=..."
2,2,ate,"V1->X,V1->Y,V2->X,X->Y",E[Y|do(X = 1)] − E[Y|do(X = 0)],P(Y=1|do(X=1)) - P(Y=1|do(X=1)),[P(Y=1|V2=1)-P(Y=1|V2=0)]/[P(X=1|V2=1)-P(X=1|V...,[0.39733441500000005-0.44394854770000003]/[0.9...,-0.1227,no,"[P(Y=1|V2=0)=0.44394854770000003, P(Y=1|V2=1)=..."
3,3,ate,"V1->X,V1->Y,V2->X,X->Y",E[Y|do(X = 1)] − E[Y|do(X = 0)],P(Y=1|do(X=1)) - P(Y=1|do(X=1)),[P(Y=1|V2=1)-P(Y=1|V2=0)]/[P(X=1|V2=1)-P(X=1|V...,[0.2000484216-0.3129399888]/[0.8-0.5],-0.3763,no,"[P(Y=1|V2=0)=0.3129399888, P(Y=1|V2=1)=0.20004..."
4,4,ate,"V1->X,V1->Y,V2->X,X->Y",E[Y|do(X = 1)] − E[Y|do(X = 0)],P(Y=1|do(X=1)) - P(Y=1|do(X=1)),[P(Y=1|V2=1)-P(Y=1|V2=0)]/[P(X=1|V2=1)-P(X=1|V...,[0.5671622485-0.6026637459]/[0.2-0.14],-0.5917,no,"[P(Y=1|V2=0)=0.6026637459, P(Y=1|V2=1)=0.56716..."


In [86]:
# Drop rows with null values in step0, step1, step2, step3, step4
print(len(df))
df = df.dropna(subset=["step0", "step1", "step2", "step3", "step4"])
print(len(df))

10686
5817


In [119]:
# Fill the reasoning_template with values from the dataframe
gt_reasonings = []
for index, row in df.iterrows():
    # Join the elements in the 'given_info_formal' column into a string
    given_info_formal_string = '; '.join(row['given_info_formal'])

    # Replace ' - ' with ' and ' in 'step1'
    step1_replaced = row['step1'].replace(' - ', ' and ')

    # Create a dictionary from the row
    row_dict = row.to_dict()

    # Replace the 'given_info_formal' with the joined string
    row_dict['given_info_formal'] = given_info_formal_string

    # Replace the 'step1' with the replaced string
    row_dict['step1_format'] = step1_replaced

    # Now, format the template
    reasoning_template_filled = reasoning_template.format(**row_dict)
    # make dict of ID and reasoning_template_filled
    gt_reasonings.append({"ID": row_dict["ID"], "gt_reasoning": reasoning_template_filled})
print(gt_reasonings[0])

{'ID': 0, 'gt_reasoning': 'Step 1) Extract the causal graph: The causal graph expressed in the context is: "V1->X,V1->Y,V2->X,X->Y".\n\nStep 2) Identify the query type: The query type of the above question is "ate".\n\nStep 3) Translate the query to an estimand: Based on the type of the causal query, the question can be translated to a formal estimand as: "P(Y=1|do(X=1)) - P(Y=1|do(X=1)) ".\n\nStep 4) Collect all the available data: The available data are: "P(Y=1|V2=0)=0.6026637459; P(Y=1|V2=1)=0.5671622485; P(X=1|V2=0)=0.14; P(X=1|V2=1)=0.2".\n\nStep 5) Solve for the estimand: To solve for the estimand, we need to compute P(Y=1|do(X=1)) and P(Y=1|do(X=1)) .\n\nUsing the causal graph, we can simplify the estimand to\nP(Y=1|do(X=1)) - P(Y=1|do(X=1)) \n= [P(Y=1|V2=1)-P(Y=1|V2=0)]/[P(X=1|V2=1)-P(X=1|V2=0)]\n\nUsing the available data, we can calculate:\n[P(Y=1|V2=1)-P(Y=1|V2=0)]/[P(X=1|V2=1)-P(X=1|V2=0)]\n= [0.5671622485-0.6026637459]/[0.2-0.14]\n≈ -0.5917\n\nSince the estimate for the es

In [123]:
# Save the gt_reasonings as a jsonl file
with open("raw_datasets/id2truth_reasoning.jsonl", "w") as f:
    for item in gt_reasonings:
        f.write(json.dumps(item) + "\n")



In [124]:
# Read the jsonl as a pandas dataframe
gt_reasonings_df = pd.read_json("raw_datasets/id2truth_reasoning.jsonl", lines=True)
gt_reasonings_df.head()

,ID,gt_reasoning
0,0,Step 1) Extract the causal graph: The causal g...
1,1,Step 1) Extract the causal graph: The causal g...
2,2,Step 1) Extract the causal graph: The causal g...
3,3,Step 1) Extract the causal graph: The causal g...
4,4,Step 1) Extract the causal graph: The causal g...


In [136]:
# Merge overall_result_no_null and gt_reasonings_df on ID
overall_result_gt = overall_result_no_null.merge(gt_reasonings_df, on="ID")
overall_result_gt.head()

,ID,prompt,pred,gt_reasoning
0,4669,"Imagine a self-contained, hypothetical world w...",Step 1) Extract the causal graph: The causal g...,Step 1) Extract the causal graph: The causal g...
1,1597,"Imagine a self-contained, hypothetical world w...",Step 1) Extract the causal graph: The causal g...,Step 1) Extract the causal graph: The causal g...
2,2995,"Imagine a self-contained, hypothetical world w...",Step 1) Extract the causal graph: The causal g...,Step 1) Extract the causal graph: The causal g...
3,9475,"Imagine a self-contained, hypothetical world w...",Step 1) Extract the causal graph: The causal g...,Step 1) Extract the causal graph: The causal g...
4,6328,"Imagine a self-contained, hypothetical world w...",Step 1) Extract the causal graph: The causal g...,Step 1) Extract the causal graph: The causal g...


In [139]:
# Check if there are empty entries in any column
overall_result_gt.isnull().sum()

ID              0
prompt          0
pred            0
gt_reasoning    0
dtype: int64

## Formatting for ROSCOE

In [24]:
def prepare_json_for_roscoe(
    df: pd.DataFrame,
    prompt_column: str,
    gpt_column: str,
    out_name: str,
    gt_column: Union[str, None] = None,
    id_column: Union[str, None] = None,
) -> None:
    df_out = df[[prompt_column, gpt_column]]
    df_out.rename(columns={gpt_column: "gpt-3", prompt_column: "premise"}, inplace=True)
    # As strange as it sounds, hypotesis column is used for ground truth, left empty otherwise
    if gt_column is not None:
        df_out["hypothesis"] = (
            "IGNORE THIS. Ground truth here for reference. " + df[gt_column]
        )
    else:
        df_out["hypothesis"] = " "
    if id_column is not None:
        df_out["ID"] = df[id_column]

    df_out.to_json("datasets/" + out_name + ".jsonl", orient="records", lines=True)
    return None

In [25]:
prepare_json_for_roscoe(
    overall_result_no_null, "prompt", "pred", "cb1", gt_column=None, id_column="ID"
)

## Run ROSCOE

```python
python projects/roscoe/roscoe.py -t sim_sce -m facebook/roscoe-512-roberta-base --dataset-path ../CausalLLMs/roscoe_exp/datasets --datasets cb1
```